In [19]:
year = 2023
day = 19

In [20]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
# data = puzzle.examples[0].input_data

data = data.strip()
all_rules, ranges = data.split("\n\n")
all_rules = all_rules.split("\n")
ranges = ranges.split("\n")

all_rules = [d.split("{") for d in all_rules]
all_rules = {d[0]: d[1].replace("}", "").split(",") for d in all_rules}

ranges = [d.replace("{", "").replace("}", "").split(",") for d in ranges]
ranges = [[int(n.split("=")[1]) for n in d] for d in ranges]

In [21]:
def apply_rule(rule, part):
    x, m, a, s = part # used in eval()
    for r in rule:
        if ":" in r:
            if eval(r.split(":")[0]):
                return r.split(":")[1]
        else:
            return r

In [22]:
from collections import defaultdict
cache = defaultdict(list)
cache["in"] = ranges

out = defaultdict(list)
while cache:
    new_cache = defaultdict(list)
    for key, ranges in cache.items():
        new_parts = []
        for part in ranges:
            new_rule = apply_rule(all_rules[key], part)
            if new_rule in "AR":
                out[new_rule].append(part)
            else:
                new_cache[new_rule].append(part)
    cache = new_cache

In [23]:
answer = sum(sum(xmas) for xmas in out["A"])
submit(answer, part="a", year=year, day=day)

Part a already solved with same answer: 386787


In [24]:
# idea
# start with ranges
# start with 'in' rule
# split ranges based on rules
# keep track of possible ranges for each rule
# when a rule is resolved, add to the output

c_order = {"x": 0, "m": 1, "a": 2, "s": 3}

cache = defaultdict(list)
cache["in"] = [[range(1, 4001), range(1, 4001), range(1, 4001), range(1, 4001)]]
out = defaultdict(list)

MAX_ITER = 1_000
for i in range(MAX_ITER):
    new_cache = defaultdict(list)
    for key, ranges in cache.items():
        for part_range in ranges:
            for r in all_rules[key]:
                if ":" in r:
                    c, op, val, new_key = r[0], r[1], int(r.split(":")[0][2:]), r.split(":")[1]
                    if op == ">":
                        val += 1
                        new_part_range = part_range.copy()
                        new_part_range[c_order[c]] = range(val, part_range[c_order[c]].stop)
                        part_range[c_order[c]] = range(part_range[c_order[c]].start, val)
                    elif op == "<":
                        new_part_range = part_range.copy()
                        new_part_range[c_order[c]] = range(part_range[c_order[c]].start, val)
                        part_range[c_order[c]] = range(val, part_range[c_order[c]].stop)
                    else:
                        raise ValueError
                    if new_key in ['A', 'R']:
                        out[new_key].append(new_part_range)
                    else:
                        new_cache[new_key].append(new_part_range)

                elif r in ['A', 'R']:
                    out[r].append(part_range)
                else:
                    new_cache[r].append(part_range)
    if not new_cache:
        print(f"finished in {i} iterations")
        break
    cache = new_cache

finished in 6 iterations


In [25]:
import math
answer = sum(math.prod(len(r) for r in xmas_r) for xmas_r in out["A"])
submit(answer, part="b", year=year, day=day)

Part b already solved with same answer: 131029523269531
